In [32]:
import tensorflow as tf #모듈 
import numpy as np

In [151]:
def split_input_target(chunk): 
    input_text = chunk[:-1]
    output_text = chunk[1:]
    return [input_text, output_text]

In [152]:
mrugesh = open("database/mrugesh.txt")
mrugesh_train_data = [i for i in mrugesh.readline()]

In [153]:
char2idx = {'R':0,'P':1,'S':2}

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

# mrugesh_train_data = np.expand_dims(mrugesh_train_data, axis=1)[0]
mrugesh_train_data = text_to_int(mrugesh_train_data)
print(mrugesh_train_data)

mrugesh_train_data = tf.data.Dataset.from_tensor_slices(mrugesh_train_data)

[0 1 1 ... 1 1 2]


In [154]:
seq_length = 100
examples_per_epoch = len(mrugesh_train_data)//(seq_length+1)
sequences = mrugesh_train_data.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)

In [155]:
print(list(dataset.as_numpy_iterator()))

[(array([0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1,
       1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2,
       0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0,
       1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1,
       2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0]), array([1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1,
       2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0,
       0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1,
       1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2,
       0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0])), (array([1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1,
       2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0,
       0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1,
       1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 2,
       0, 0, 1, 1, 2, 0, 0, 1,

In [156]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=[batch_size, None]), 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [159]:
BATCH_SIZE = 100
VOCAB_SIZE = 3
EMBEDDING_DIM = 8
RNN_UNITS = 1024

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.build(tf.TensorShape([1]))
model.summary() 

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (100, None, 8)            24        
                                                                 
 lstm_8 (LSTM)               (100, None, 1024)         4231168   
                                                                 
 dense_8 (Dense)             (100, None, 3)            3075      
                                                                 
Total params: 4,234,267
Trainable params: 4,234,267
Non-trainable params: 0
_________________________________________________________________


In [7]:
def loss(labels, logits): #logits = probability distribution
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [160]:
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
history = model.fit(dataset, epochs=40)

Epoch 1/40


ValueError: in user code:

    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.9/site-packages/keras/engine/input_spec.py", line 214, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" '

  

In [ ]:
def player(prev_play, opponent_history=[]):
    opponent_history.append(prev_play)

    guess = "yee"
    
    return guess 
